# 🎨 Look1nce - OOTDiffusion Backend on Google Colab T4 GPU

This notebook runs OOTDiffusion model on Google Colab's FREE T4 GPU and exposes it as an API for your Look1nce app.

## 📋 Instructions:
1. Go to **Runtime** → **Change runtime type** → Select **T4 GPU** → Save
2. Run all cells in order (click ▶️ or press Shift+Enter)
3. Copy the **Gradio public URL** (looks like: https://xxxxx.gradio.live)
4. Paste it in your backend's `.env` file
5. Your React app will now use Colab GPU! 🚀

## Step 1: Setup & Install Dependencies

In [ ]:
# Check GPU availability
!nvidia-smi

print("\n✅ GPU detected! You're ready to go.")

In [ ]:
# Install required packages
print("📦 Installing dependencies... (this takes 2-3 minutes)")

!pip install -q gradio==4.44.0
!pip install -q diffusers==0.27.2
!pip install -q transformers==4.38.2
!pip install -q accelerate==0.27.2
!pip install -q opencv-python-headless
!pip install -q pillow

print("✅ All packages installed!")

## Step 2: Clone OOTDiffusion Repository & Install Requirements

In [ ]:
import os
from pathlib import Path

# Clone repo if not exists
if not Path("OOTDiffusion").exists():
    print("📥 Downloading OOTDiffusion...")
    !git clone https://github.com/levihsu/OOTDiffusion.git
    print("✅ Repository cloned!")
else:
    print("✅ Repository already exists!")

# Change to repo directory
%cd OOTDiffusion

# Install repository requirements
print("\n📦 Installing OOTDiffusion requirements...")
!pip install -q -r requirements.txt
print("✅ Requirements installed!")

## Step 3: Download Model Checkpoints (First Time Only)

In [ ]:
from huggingface_hub import snapshot_download
import os

print("📥 Downloading model weights... (this takes 5-10 minutes first time)")
print("⏳ Please wait, models are ~2-3GB...\n")

# Download checkpoints
checkpoint_path = snapshot_download(
    repo_id="levihsu/OOTDiffusion",
    local_dir="./checkpoints",
    local_dir_use_symlinks=False
)

print("\n✅ Model weights downloaded successfully!")
print(f"📁 Saved to: {checkpoint_path}")

## Step 4: Import Libraries & Setup

In [ ]:
import gradio as gr
import torch
from PIL import Image
import numpy as np
from pathlib import Path
import sys

# Add current directory to Python path
sys.path.append(str(Path.cwd()))

print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print("\n✅ Environment ready!")

## Step 5: Load OOTDiffusion Model

In [ ]:
# Import OOTDiffusion inference code
from preprocess.openpose.run_openpose import OpenPose
from preprocess.humanparsing.run_parsing import Parsing
from ootd.inference_ootd_hd import OOTDiffusionHD
from ootd.inference_ootd_dc import OOTDiffusionDC

print("🔄 Loading OOTDiffusion models...")
print("⏳ This takes 2-3 minutes...\n")

# Initialize preprocessing models
openpose_model = OpenPose(0)  # 0 = GPU device
parsing_model = Parsing(0)

# Initialize OOTDiffusion HD model (better quality)
ootd_model = OOTDiffusionHD(gpu_id=0)

print("✅ Models loaded successfully!")
print("🚀 Ready to process images!")

## Step 6: Create Try-On Function

In [ ]:
def virtual_tryon(person_img, cloth_img, category, num_samples=1, num_steps=20, seed=-1):
    """
    Run virtual try-on using OOTDiffusion
    
    Args:
        person_img: PIL Image or file path
        cloth_img: PIL Image or file path  
        category: 'Upper-body', 'Lower-body', or 'Dress'
        num_samples: Number of results to generate
        num_steps: Inference steps (20-50, higher = better quality)
        seed: Random seed (-1 for random)
    
    Returns:
        List of result images
    """
    try:
        print(f"🎨 Processing try-on request...")
        print(f"   Category: {category}")
        print(f"   Steps: {num_steps}")
        
        # Load images
        if isinstance(person_img, str):
            person_img = Image.open(person_img)
        if isinstance(cloth_img, str):
            cloth_img = Image.open(cloth_img)
        
        # Convert to RGB if needed
        person_img = person_img.convert('RGB')
        cloth_img = cloth_img.convert('RGB')
        
        # Preprocess images
        print("📐 Preprocessing person image...")
        keypoints = openpose_model(person_img)
        model_parse, _ = parsing_model(person_img)
        
        # Map category to model format
        category_map = {
            'Upper-body': 0,
            'Lower-body': 1,
            'Dress': 2
        }
        category_idx = category_map.get(category, 0)
        
        # Run try-on
        print("🔄 Running AI model... (this takes 20-40 seconds)")
        results = ootd_model(
            category=category_idx,
            image_garm=cloth_img,
            image_vton=person_img,
            mask=model_parse,
            image_ori=person_img,
            num_samples=num_samples,
            num_steps=num_steps,
            seed=seed
        )
        
        print("✅ Try-on complete!")
        return results
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        raise gr.Error(f"Try-on failed: {str(e)}")

print("✅ Try-on function ready!")

## Step 7: Create Gradio API Interface

In [ ]:
# Create Gradio interface
print("🌐 Creating API interface...\n")

with gr.Blocks(title="Look1nce - OOTDiffusion API", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎨 Look1nce Virtual Try-On API
    
    ### Running on Google Colab T4 GPU ⚡
    
    This is the backend API for your Look1nce app. Keep this tab open while using your app!
    """)
    
    with gr.Row():
        with gr.Column():
            person_input = gr.Image(label="👤 Person Photo", type="pil")
            cloth_input = gr.Image(label="👔 Clothing Photo", type="pil")
        
        with gr.Column():
            result_output = gr.Gallery(label="✨ Try-On Results", columns=1)
    
    with gr.Row():
        category_input = gr.Dropdown(
            choices=['Upper-body', 'Lower-body', 'Dress'],
            value='Upper-body',
            label="👕 Clothing Category"
        )
        steps_input = gr.Slider(
            minimum=10,
            maximum=50,
            value=20,
            step=5,
            label="🎯 Quality Steps (higher = better, slower)"
        )
    
    submit_btn = gr.Button("🚀 Generate Try-On", variant="primary", size="lg")
    
    # Connect function
    submit_btn.click(
        fn=virtual_tryon,
        inputs=[person_input, cloth_input, category_input, gr.Number(value=1, visible=False), steps_input, gr.Number(value=-1, visible=False)],
        outputs=result_output
    )
    
    gr.Markdown("""
    ---
    ### 📝 API Usage:
    - This interface can be called from your Look1nce backend
    - Copy the public URL above and paste it in your `.env` file
    - Your React frontend → Backend → Colab API → Results!
    """)

print("✅ Interface created!")

## Step 8: 🚀 Launch API Server

In [ ]:
print("="*60)
print("🚀 LAUNCHING LOOK1NCE API SERVER")
print("="*60)
print("\n⏳ Starting Gradio server...\n")

# Launch with public URL
demo.launch(
    share=True,           # Create public URL
    server_name="0.0.0.0",
    server_port=7860,
    debug=True,
    show_error=True
)

print("\n" + "="*60)
print("✅ SERVER IS RUNNING!")
print("="*60)
print("\n📋 NEXT STEPS:")
print("1. Copy the 'public URL' above (https://xxxxx.gradio.live)")
print("2. Open your backend folder: D:/Look1nce/backend")
print("3. Edit the .env file and add: COLAB_API_URL=<your-url>")
print("4. Restart your backend server")
print("5. Your React app will now use Colab GPU! 🎉")
print("\n⚠️ IMPORTANT: Keep this tab open while using your app!")
print("\n" + "="*60)

---

## 🎉 You're All Set!

### What's Happening:
- ✅ OOTDiffusion is running on Colab T4 GPU
- ✅ Gradio created a public API URL
- ✅ Your React app can now send images to Colab
- ✅ Results come back in ~20-40 seconds

### Limitations:
- ⏱️ Session expires after ~12 hours of inactivity
- 🔄 Just re-run all cells to restart
- 💰 FREE tier: ~10-20 hours GPU/day
- ⭐ Colab Pro ($10/month): 100+ hours GPU/month

### Troubleshooting:
- If session disconnects: Just re-run all cells
- If out of GPU quota: Wait 24 hours or upgrade to Colab Pro
- If errors: Check the error messages and restart runtime

---

**Made with ❤️ for Look1nce**